<a href="https://colab.research.google.com/github/moh2236945/Car_spots_detector/blob/master/Segmentation_of_ID_Cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
!mkdir dataset

In [2]:
#This script just downloads the dataset and stores the both images and their respective masks
#in numpy file. There will be 50 .npy files, you can use them seperately or can run other script
#to combine all of them in single file.


import os
import shutil
import cv2
import json
import numpy as np 
from glob import glob

all_links = ['ftp://smartengines.com/midv-500/dataset/01_alb_id.zip',
             'ftp://smartengines.com/midv-500/dataset/02_aut_drvlic_new.zip',
             'ftp://smartengines.com/midv-500/dataset/03_aut_id_old.zip',
             'ftp://smartengines.com/midv-500/dataset/04_aut_id.zip',
             'ftp://smartengines.com/midv-500/dataset/05_aze_passport.zip', 
             'ftp://smartengines.com/midv-500/dataset/06_bra_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/07_chl_id.zip',
             'ftp://smartengines.com/midv-500/dataset/08_chn_homereturn.zip',
             'ftp://smartengines.com/midv-500/dataset/09_chn_id.zip',
             'ftp://smartengines.com/midv-500/dataset/10_cze_id.zip',
             'ftp://smartengines.com/midv-500/dataset/11_cze_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/12_deu_drvlic_new.zip',
             'ftp://smartengines.com/midv-500/dataset/13_deu_drvlic_old.zip',
             'ftp://smartengines.com/midv-500/dataset/14_deu_id_new.zip',
             'ftp://smartengines.com/midv-500/dataset/15_deu_id_old.zip',
             'ftp://smartengines.com/midv-500/dataset/16_deu_passport_new.zip',
             'ftp://smartengines.com/midv-500/dataset/17_deu_passport_old.zip',
             'ftp://smartengines.com/midv-500/dataset/18_dza_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/19_esp_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/20_esp_id_new.zip',
             'ftp://smartengines.com/midv-500/dataset/21_esp_id_old.zip',
             'ftp://smartengines.com/midv-500/dataset/22_est_id.zip',
             'ftp://smartengines.com/midv-500/dataset/23_fin_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/24_fin_id.zip',
             'ftp://smartengines.com/midv-500/dataset/25_grc_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/26_hrv_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/27_hrv_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/28_hun_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/29_irn_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/30_ita_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/31_jpn_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/32_lva_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/33_mac_id.zip',
             'ftp://smartengines.com/midv-500/dataset/34_mda_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/35_nor_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/36_pol_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/37_prt_id.zip',
             'ftp://smartengines.com/midv-500/dataset/38_rou_drvlic.zip',
             'ftp://smartengines.com/midv-500/dataset/39_rus_internalpassport.zip',
             'ftp://smartengines.com/midv-500/dataset/40_srb_id.zip',
             'ftp://smartengines.com/midv-500/dataset/41_srb_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/42_svk_id.zip',
             'ftp://smartengines.com/midv-500/dataset/43_tur_id.zip',
             'ftp://smartengines.com/midv-500/dataset/44_ukr_id.zip',
             'ftp://smartengines.com/midv-500/dataset/45_ukr_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/46_ury_passport.zip',
             'ftp://smartengines.com/midv-500/dataset/47_usa_bordercrossing.zip',
             'ftp://smartengines.com/midv-500/dataset/48_usa_passportcard.zip',
             'ftp://smartengines.com/midv-500/dataset/49_usa_ssn82.zip',
             'ftp://smartengines.com/midv-500/dataset/50_xpo_id.zip']

def read_image(img, label):
    image = cv2.imread(img)
    mask = np.zeros(image.shape, dtype=np.uint8)
    quad = json.load(open(label, 'r'))
    coords = np.array(quad['quad'], dtype=np.int32)
    cv2.fillPoly(mask, coords.reshape(-1, 4, 2), color=(255,255,255))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.resize(mask, (mask.shape[1]//2, mask.shape[0]//2))
    image = cv2.resize(image, (image.shape[1]//2, image.shape[0]//2))
    mask = cv2.threshold(mask, 0,255, cv2.THRESH_BINARY)[1]
    mask = cv2.resize(mask, (256,256))
    image = cv2.resize(image, (256, 256))
    return image, mask


def main():
    i = 1

    for link in all_links:
        print('----------------------------------------------------------------------')
        print('\nDownloading:', link[40:])
        os.system('wget ' + link)
        print('Downloaded:', link[40:])
        print('Unzipping:', link[40:])
        os.system('unzip ' + link[40:])
        print('Unzipped:', link[40:].replace('.zip', ''))

        imgdir_path = './' + link[40:].replace('.zip', '') + '/images/'
        crddir_path = './' + link[40:].replace('.zip', '') + '/ground_truth/'
        # time.sleep(5)
        X = []
        Y = []
        os.remove(imgdir_path + link[40:].replace('.zip', '.tif'))
        os.remove(crddir_path + link[40:].replace('.zip', '.json'))
        for images, ground_truth in zip(sorted(os.listdir(imgdir_path)), sorted(os.listdir(crddir_path))):
            img_list = sorted(glob(imgdir_path + images + '/*.tif'))
            label_list = sorted(glob(crddir_path + ground_truth + '/*.json'))
            for img, label in zip(img_list, label_list):
                image, mask = read_image(img, label)
                X.append(image)
                Y.append(mask)

        X = np.array(X)
        Y = np.array(Y)
        Y = np.expand_dims(Y, axis=3)
        print(link[40:].replace('.zip', ''), X.shape, Y.shape)
        # print(X.shape, Y.shape)
        np.save('train_image' + str(i) + '.npy', X)
        np.save('mask_image' + str(i) + '.npy', Y)
        print('Files Saved For:', link[40:].replace('.zip', ''))
        i += 1
        print('----------------------------------------------------------------------')
        os.remove(link[40:])
        shutil.rmtree(link[40:].replace('.zip', ''))

if __name__ == '__main__':
    main()

----------------------------------------------------------------------

Downloading: 01_alb_id.zip
Downloaded: 01_alb_id.zip
Unzipping: 01_alb_id.zip
Unzipped: 01_alb_id
01_alb_id (300, 256, 256, 3) (300, 256, 256, 1)
Files Saved For: 01_alb_id
----------------------------------------------------------------------
----------------------------------------------------------------------

Downloading: 02_aut_drvlic_new.zip
Downloaded: 02_aut_drvlic_new.zip
Unzipping: 02_aut_drvlic_new.zip
Unzipped: 02_aut_drvlic_new
02_aut_drvlic_new (300, 256, 256, 3) (300, 256, 256, 1)
Files Saved For: 02_aut_drvlic_new
----------------------------------------------------------------------
----------------------------------------------------------------------

Downloading: 03_aut_id_old.zip
Downloaded: 03_aut_id_old.zip
Unzipping: 03_aut_id_old.zip
Unzipped: 03_aut_id_old
03_aut_id_old (300, 256, 256, 3) (300, 256, 256, 1)
Files Saved For: 03_aut_id_old
---------------------------------------------------

In [0]:
#This script stacks all files and create a single file

import numpy as np

def main():
    total = np.load('train_image1.npy')
    for i in range(2,51):
        temp = np.load('train_image' + str(i) + '.npy')
        total = np.vstack((total, temp))

    np.save('final_train.npy', total)

    total = np.load('mask_image1.npy')
    for i in range(2,51):
        temp = np.load('mask_image' + str(i) + '.npy')
        total = np.vstack((total, temp))
    np.save('final_mask.npy', total)


if __name__ == '__main__':
    main()



In [0]:
import numpy as np
from keras import backend as K

def IoU(y_true, y_pred, eps=1e-6):
    if np.max(y_true) == 0.0:
        return IoU(1-y_true, 1-y_pred) ## empty image; calc IoU of zeros
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return -K.mean( (intersection + eps) / (union + eps), axis=0)

In [0]:
from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPool2D, Input, concatenate, UpSampling2D, Dropout
from keras.layers import Conv2DTranspose, AvgPool2D, GaussianNoise, BatchNormalization
import tensorflow as tf
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import keras
from keras.callbacks import TensorBoard, ModelCheckpoint


def get_model():
    K.clear_session()
    inputs = Input((256,256,3))
    s = BatchNormalization()(inputs) # we can learn the normalization step
    s = Dropout(0.5)(s)

    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPool2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPool2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPool2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPool2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
    c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer=Adam(lr=1e-4), loss=IoU, metrics=['binary_accuracy'])

    return model

In [10]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, ModelCheckpoint

X = np.load('final_train.npy')
Y = np.load('final_mask.npy')

def t_generator(X_train, Y_train, batch_size):
  features = np.zeros(shape=(batch_size, 256,256,3))
  labels = np.zeros(shape=(batch_size, 256,256,1))
  while True:
    start = 0
    end = batch_size
    for i in range(243):
      features = X_train[start:end]
      labels = Y_train[start:end]
      start = end
      end = end + batch_size
      yield features / 255.0, labels / 255.0
      
def v_generator(X_val, Y_val, batch_size):
  features = np.zeros(shape=(batch_size, 256,256,3))
  labels = np.zeros(shape=(batch_size, 256,256,1))
  while True:
    start = 0
    end = batch_size
    for i in range(27):
      features = X_val[start:end]
      labels = Y_val[start:end]
      start = end
      end = end + batch_size
      yield features / 255.0, labels / 255.0

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    shuffle=True, 
                                                    random_state=265, 
                                                    test_size=0.1)
del X, Y
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, 
                                                  shuffle=True, 
                                                  random_state=265,
                                                  test_size=0.1)

model = get_model()

## Adding Callbacks
tb = TensorBoard(log_dir='./logs/', batch_size=8, write_graph=True)


model_chkpt = ModelCheckpoint('unet_model_whole_100epochs.h5',
                              monitor='val_loss', verbose=1, 
                              save_best_only=True)


history = model.fit_generator(generator = t_generator(X_train, Y_train, 50), 
                              steps_per_epoch=X_train.shape[0] // 50, 
                              epochs=100, callbacks=[tb, model_chkpt], 
                              validation_data=v_generator(X_val, Y_val, 50),
                              validation_steps=X_val.shape[0] // 50)












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 256, 3)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224       

NotImplementedError: ignored